In [1]:
import math
import cmath
import numpy as np
import pandas as pd
from numpy.linalg import svd
from numpy.random import uniform
from scipy import io
from scipy.linalg import eigh
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

%matplotlib qt
sns.set()
axis_font = {'size':'16'}
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16

Load the data files. The 8 non-empty strings are mapped to the unit circle in the complex plane separated by 45 degrees and '--' are mapped to the origin. In this way we hope that the 'pure' patters are separated by the 'hybrid' patters, and '--' is not biased towards any non-empty pattern. Each column of $X$ are shifted to have zero mean.

In [2]:
data = io.loadmat('genomedata.mat')
map_gen = {'AA': complex(0.0, 1.0),
           'CC': complex(-1.0, 0.0),
           'GG': complex(1.0, 0.0),
           'TT': complex(0.0, -1.0),
           'AG': complex(1.0 / math.sqrt(2), 1.0 / math.sqrt(2)),
           'AC': complex(-1.0 / math.sqrt(2), 1.0 / math.sqrt(2)),
           'TC': complex(-1.0 / math.sqrt(2), -1.0 / math.sqrt(2)),
           'TG': complex(1.0 / math.sqrt(2), -1.0 / math.sqrt(2)), 
           '--': complex(0.0, 0.0)}

X_raw = [[string.strip() for string in row[0][0].strip().split('\t')] for row in data['X']]
X = np.array([[map_gen[string] for string in row] for row in X_raw])
n, p = X.shape
X = X - np.mean(X, axis=0) * np.ones(p)

random_state = 8 # The random state used by the KMeans

To select the parameter $k$ of the k-means properly we define the following function to evaluate the gap statistic

In [3]:
# The function to evaluate the gap statistic for k-means with different number of clusters
def get_gapstat(X, k_range=np.arange(1, 9), n_sample=20, random_state=8):
    n, p = X.shape
    
    X_ref = np.empty((n, p, n_sample), dtype='float64')
    for col in range(p):
        X_ref[:, col, :] = uniform(low=X[:, col].min(), high=X[:, col].max(), size=(n, n_sample))
    
    gapstat = np.empty((n_sample, len(k_range)), dtype='float64')
    for i_k, k in enumerate(k_range):
        km = KMeans(n_clusters=k, random_state=random_state)
        
        # The inertia of the actual data
        km.fit(X)
        w = km.inertia_
        
        # The inertia of the reference data
        w_ref = np.empty(n_sample, dtype='float64')
        for i_sample in range(n_sample):
            km.fit(X_ref[:, :, i_sample])
            w_ref[i_sample] = km.inertia_
        
        gapstat[:, i_k] = np.log(w_ref) - np.log(w)
        
    return gapstat

k_range = np.arange(1, 9)

Method 1: Dimension reduction by PCA.

In [4]:
d_pca = 2 # For PCA we reduce the dimension of X down to 3
k_pca = 2 # The expected number of clusters for PCA

_, s_pca, V_pca = svd(X) # svd of X. Note that this is different from matlab in that X = U diag(s) V.
V_pca = np.conjugate(V_pca.T)

X_pca_complex = np.dot(X, V_pca[:, 0 : d_pca]) # Reduce the dimension of X by linear projection, a complex matrix

# Since KMeans does not seem to support complex value, expand the space to 2*d_pca dim
X_pca = np.empty((n, 2 * d_pca), dtype="float64") 
X_pca[:, 0::2] = X_pca_complex.real
X_pca[:, 1::2] = X_pca_complex.imag

label_pca = KMeans(n_clusters=2, random_state=random_state).fit_predict(X_pca)
gap_pca = get_gapstat(X_pca, k_range=k_range)

Plot the distribution of the singular value, and the scattermatrix plot of the reduced dimension data and the gap statistic for different $k$.

In [5]:
# Distribution of the sinular values
fig, ax = plt.subplots(figsize=(7, 6))
axins = inset_axes(ax, 2, 3, loc=1) # zoom-factor: 2.5, location: upper-left

ax.semilogy(np.arange(p), s_pca, linewidth=2)
ax.grid(True)
ax.set_xlabel('dimension', **axis_font)
ax.set_ylabel('singular value', **axis_font)

axins.semilogy(np.arange(p), s_pca, linewidth=2)
axins.grid(True)
x1, x2, y1, y2 = 0, 20, 50, 2000 # specify the limits
axins.set_xlim(x1, x2) # apply the x-limits
axins.set_ylim(y1, y2) # apply the y-limits
axins.set_xticks(np.arange(0, 20, 5))
mark_inset(ax, axins, loc1=2, loc2=3, fc="none", ec="0.5", lw=2)

fig.savefig('pca_singular.pdf', dpi=10)

# Scatterplot matrix
df_pca = pd.DataFrame({**{'cluster': label_pca}, **{col+1: X_pca[:, col] for col in range(2 * d_pca)}})
sns.pairplot(df_pca, hue='cluster', vars=np.arange(1, 1 + 2 * d_pca))
plt.savefig('pca_scatter.pdf', dpi=10)

# Gap statistics
fig, ax = plt.subplots(figsize=(7, 6))
ax.errorbar(k_range, np.mean(gap_pca, axis=0), yerr=np.std(gap_pca, axis=0))
ax.grid(True)
ax.set_xlabel('$k$', **axis_font)
ax.set_ylabel('Gap', **axis_font)
fig.savefig('pca_gap.pdf', dpi=10)

Method 2: Dimension reduction by diffusion maps and/or spectral clustering.

In [6]:
d_dif = 3 # For PCA we reduce the dimension of X down to 3 for 3-D visualization
k_dif = 2 # The expected number of clusters for diffusion map
t_dif = 20 # The time scale for diffusion map
f = 10 # The factor used to set epsilon according to the median of the square pairwise Euclidean distance

distance = pdist(np.concatenate((X.real, X.imag), axis=1), 'euclidean');
epsilon = np.median(distance ** 2) / f;
W = np.exp(-squareform(distance) ** 2 / epsilon)
D_inv_sqrt = np.diag(1 / np.sqrt(np.sum(W, axis=1)))
MS = np.dot(np.dot(D_inv_sqrt, W), D_inv_sqrt) # The symmetric M matrix

l_dif, V_dif = eigh(MS, eigvals=(n-d_dif-1, n-2)) # Remember to ignore the largest eigen vector
l_dif, V_dif = l_dif[::-1], V_dif[:, ::-1] # The eigen value are now in descending order. 

X_dif = np.dot(np.dot(D_inv_sqrt, V_dif), np.diag(l_dif ** t_dif)) # The redued dimension map at time t

label_dif = KMeans(n_clusters=k_dif, random_state=random_state).fit_predict(X_dif)
gap_dif = get_gapstat(X_dif, k_range=k_range)

The scattermatrix plot, the 3-D scatter plot of the reduced-dimension data and the Gap statistic.

In [8]:
df_dif = pd.DataFrame({**{'cluster': label_dif}, **{col+1: X_dif[:, col] for col in range(d_dif)}})
plot_scatterplot = sns.pairplot(df_dif, hue='cluster', vars=np.arange(1, 1 + d_dif))
plt.savefig('dif_scatter.pdf', dpi=10)

fig = plt.figure(1, figsize=(7, 6))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
plt.cla()

y = np.choose(label_dif, ['b', 'g'])
ax.scatter(X_dif[:, 0], X_dif[:, 1], X_dif[:, 2], c=y)
ax.set_xlabel('1', **axis_font)
ax.set_ylabel('2', **axis_font)
ax.set_zlabel('3', **axis_font)
fig.savefig('dif_3d.pdf', dpi=10)

# Gap statistics
fig, ax = plt.subplots(figsize=(7, 6))
ax.errorbar(k_range, np.mean(gap_dif, axis=0), yerr=np.std(gap_dif, axis=0))
ax.grid(True)
ax.set_xlabel('$k$', **axis_font)
ax.set_ylabel('Gap', **axis_font)

fig.savefig('dif_gap.pdf', dpi=10)

Spectral clustering method.

In [10]:
k_spec = 5 # The expected number of clusters for spectral clustering

L = np.diag(sum(W)) - W
_, V_spec = eigh(L)
X_spec = V_spec[:, 1:2] # The reduced dimension data, simply the second smallest eigen vector of L
label_spec = KMeans(n_clusters=k_spec, random_state=random_state).fit_predict(X_spec)
gap_spec = get_gapstat(X_spec, k_range=k_range)

The histogram of the second smallest eigen vector of $L$ and the Gap statistic.

In [11]:
fig, ax = plt.subplots(figsize=(7, 6))

c = ['b', 'g', 'r', 'purple', 'orange']
for k in range(k_spec):
    ax.hist(X_spec[label_spec==k, 0], 20, facecolor=c[k], alpha=0.75, label='cluster {0}'.format(k))
ax.legend(loc=1, fontsize=16)
ax.set_xlabel('$v$', **axis_font)
ax.set_ylabel('count', **axis_font)
fig.savefig('spec_hist.pdf', dpi=10)

# Gap statistics
fig, ax = plt.subplots(figsize=(7, 6))
ax.errorbar(k_range, np.mean(gap_spec, axis=0), yerr=np.std(gap_spec, axis=0))
ax.grid(True)
ax.set_xlabel('$k$', **axis_font)
ax.set_ylabel('Gap', **axis_font)
fig.savefig('spec_gap.pdf', dpi=10)